### Download the necessary modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

### Import the data in excel file and format them

In [165]:
df_raw =  pd.read_excel('séries_AirLiquide.xlsx')
df_raw.head(10)

,Unnamed: 0,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40
0,1987M03,0.305684,11.1670,NaN
1,1987M04,0.306773,11.8679,NaN
2,1987M05,0.307862,11.3422,NaN
3,1987M06,0.308952,11.0714,NaN
4,1987M07,0.310043,10.9918,NaN
5,1987M08,0.311138,11.1511,1483.02
6,1987M09,0.312236,11.0873,1524.00
7,1987M10,0.313339,10.8962,1456.88
8,1987M11,0.314448,8.5226,1116.47
9,1987M12,0.315563,7.9969,1026.98


In [166]:
df = df_raw
df.head(10)

,Unnamed: 0,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40
0,1987M03,0.305684,11.1670,NaN
1,1987M04,0.306773,11.8679,NaN
2,1987M05,0.307862,11.3422,NaN
3,1987M06,0.308952,11.0714,NaN
4,1987M07,0.310043,10.9918,NaN
5,1987M08,0.311138,11.1511,1483.02
6,1987M09,0.312236,11.0873,1524.00
7,1987M10,0.313339,10.8962,1456.88
8,1987M11,0.314448,8.5226,1116.47
9,1987M12,0.315563,7.9969,1026.98


In [167]:
df['year']=[x.split("M")[0] for x in df['Unnamed: 0']]
df['month']=[x.split("M")[1] for x in df['Unnamed: 0']]
df.head(10)

,Unnamed: 0,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40,year,month
0,1987M03,0.305684,11.1670,NaN,1987,03
1,1987M04,0.306773,11.8679,NaN,1987,04
2,1987M05,0.307862,11.3422,NaN,1987,05
3,1987M06,0.308952,11.0714,NaN,1987,06
4,1987M07,0.310043,10.9918,NaN,1987,07
5,1987M08,0.311138,11.1511,1483.02,1987,08
6,1987M09,0.312236,11.0873,1524.00,1987,09
7,1987M10,0.313339,10.8962,1456.88,1987,10
8,1987M11,0.314448,8.5226,1116.47,1987,11
9,1987M12,0.315563,7.9969,1026.98,1987,12


In [168]:
df['log_return_price'] = np.log(df['Prix Airliquide']/df['Prix Airliquide'].shift(1))
df['log_return_CAC40'] = np.log(df['Indice CAC40']/df['Indice CAC40'].shift(1))
df

,Unnamed: 0,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40,year,month,log_return_price,log_return_CAC40
0,1987M03,0.305684,11.1670,NaN,1987,03,NaN,NaN
1,1987M04,0.306773,11.8679,NaN,1987,04,0.060874,NaN
2,1987M05,0.307862,11.3422,NaN,1987,05,-0.045307,NaN
3,1987M06,0.308952,11.0714,NaN,1987,06,-0.024165,NaN
4,1987M07,0.310043,10.9918,NaN,1987,07,-0.007216,NaN
...,...,...,...,...,...,...,...,...
392,2019M11,2.465649,118.4500,5761.89,2019,11,0.041088,0.060659
393,2019M12,2.474304,119.6000,5786.74,2019,12,0.009662,0.004304
394,2020M01,2.482985,126.7000,6041.50,2020,01,0.057669,0.043083
395,2020M02,2.491688,131.9000,5832.51,2020,02,0.040222,-0.035205


In [169]:
#COVAR(O3:O394;$Y$3:$Y$394)/VARP($Y$3:$Y$394)
year = '2020'
ret_price_year = df['log_return_price'][df['year'] == year]
ret_CAC_40_year = df['log_return_CAC40'][df['year'] == year]

x = ret_price_year.values
y = ret_CAC_40_year.values

matrix_xy = np.stack((x, y), axis=0)
print(ret_price_year)
print(ret_CAC_40_year)
matrix_xy

394    0.057669
395    0.040222
396   -0.053330
Name: log_return_price, dtype: float64
394    0.043083
395   -0.035205
396   -0.089436
Name: log_return_CAC40, dtype: float64


array([[ 0.05766925,  0.04022197, -0.0533304 ],
       [ 0.04308323, -0.03520489, -0.08943601]])

In [170]:
beta = []
years = df['year'].unique()
for year in years :
    ret_price_year = df['log_return_price'][df['year'] == year]
    ret_CAC_40_year = df['log_return_CAC40'][df['year'] == year]

    x = ret_price_year.values
    y = ret_CAC_40_year.values

    matrix_xy = np.stack((x, y), axis=0)
    
    var_y = np.var(y)
    cov_matrix = np.cov(matrix_xy)
    cov_xy = cov_matrix[1,0]
    beta_xy = cov_xy/var_y
    
    beta.append(beta_xy)
    
print(len(beta))
print(len(years))

34
34


In [171]:
dividend_yearly = df.groupby('year')['Div AirLiquide (lissé)'].mean().round(3)
price_yearly = df.groupby('year')['Prix Airliquide'].mean().round(3)
IndiceCAC40_yearly = df.groupby('year')['Indice CAC40'].mean().round(3)

In [172]:
df_yearly = pd.concat([dividend_yearly, price_yearly, IndiceCAC40_yearly], axis=1)
df_yearly['beta'] = beta
df_yearly.head(10)

,Div AirLiquide (lissé),Prix Airliquide,Indice CAC40,beta
year,,,,
1987,0.311,10.609,1321.470,NaN
1988,0.323,8.497,1229.658,1.170711
1989,0.336,10.926,1750.667,1.544345
1990,0.350,12.387,1859.954,0.898183
1991,0.371,12.834,1764.264,0.803905
1992,0.398,14.827,1846.913,0.500614
1993,0.425,16.328,2012.700,0.904070
1994,0.448,17.275,2065.525,0.845330
1995,0.469,18.181,1869.042,0.381206


In [173]:
df_yearly['Dividend'] = df_yearly['Div AirLiquide (lissé)']
df_yearly['Price'] = df_yearly['Prix Airliquide']
df_yearly['CAC40_basis_points'] = df_yearly['Indice CAC40']
df_yearly.drop(columns = ['Div AirLiquide (lissé)','Prix Airliquide','Indice CAC40'],inplace=True)
df_yearly

,beta,Dividend,Price,CAC40_basis_points
year,,,,
1987,NaN,0.311,10.609,1321.470
1988,1.170711,0.323,8.497,1229.658
1989,1.544345,0.336,10.926,1750.667
1990,0.898183,0.350,12.387,1859.954
1991,0.803905,0.371,12.834,1764.264
1992,0.500614,0.398,14.827,1846.913
1993,0.904070,0.425,16.328,2012.700
1994,0.845330,0.448,17.275,2065.525
1995,0.381206,0.469,18.181,1869.042


In [174]:
# Year 1987 should be removed because there is not enough data for the CAC 40 index
df_yearly.drop(['1987'], axis=0, inplace=True)
df_yearly 

,beta,Dividend,Price,CAC40_basis_points
year,,,,
1988,1.170711,0.323,8.497,1229.658
1989,1.544345,0.336,10.926,1750.667
1990,0.898183,0.350,12.387,1859.954
1991,0.803905,0.371,12.834,1764.264
1992,0.500614,0.398,14.827,1846.913
1993,0.904070,0.425,16.328,2012.700
1994,0.845330,0.448,17.275,2065.525
1995,0.381206,0.469,18.181,1869.042
1996,0.786398,0.492,20.877,2082.438


In [175]:
french_treasuries_bonds =  pd.read_excel('tx_dintéret_OAT.xlsx',sheet_name = 'Final', header = None)
french_treasuries_bonds.columns = ['year','french_bonds_rate']
french_treasuries_bonds['year'] = french_treasuries_bonds['year'].astype(str)
french_treasuries_bonds.set_index('year', inplace = True)
french_treasuries_bonds.head(10)

,french_bonds_rate
year,
1999,3.410000
2000,0.700000
2001,3.280000
2002,3.310000
2003,2.450000
2004,2.370000
2005,4.210000
2006,2.910000
2007,1.875000


### Now we need to cross the intervals of time with data available from both dataframes. in our case, the data are fine between 1999 and 2020

In [176]:
french_treasuries_bonds.drop(['2021', '2022'], axis = 0, inplace = True)
french_treasuries_bonds

,french_bonds_rate
year,
1999,3.410000
2000,0.700000
2001,3.280000
2002,3.310000
2003,2.450000
2004,2.370000
2005,4.210000
2006,2.910000
2007,1.875000


In [185]:
df_yearly.iloc[:11,:].index

Index(['1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998'],
      dtype='object', name='year')

In [186]:
df_yearly.drop(df_yearly.iloc[:11,:].index, axis = 0, inplace = True)
df_yearly

,beta,Dividend,Price,CAC40_basis_points
year,,,,
1999,0.750548,0.655,27.822,4464.609
2000,0.150562,0.752,29.066,6353.173
2001,-0.121145,0.870,32.785,5085.517
2002,0.421490,0.979,33.140,3866.780
2003,0.622865,1.038,30.311,3115.573
2004,0.807549,1.027,33.690,3699.040
2005,0.806305,0.988,37.058,4247.107
2006,1.038258,1.009,44.525,5095.468
2007,0.505131,1.103,52.345,5752.749


In [189]:
df_yearly_final = pd.concat([df_yearly,french_treasuries_bonds], axis = 1)
df_yearly_final

,beta,Dividend,Price,CAC40_basis_points,french_bonds_rate
year,,,,,
1999,0.750548,0.655,27.822,4464.609,3.410000
2000,0.150562,0.752,29.066,6353.173,0.700000
2001,-0.121145,0.870,32.785,5085.517,3.280000
2002,0.421490,0.979,33.140,3866.780,3.310000
2003,0.622865,1.038,30.311,3115.573,2.450000
2004,0.807549,1.027,33.690,3699.040,2.370000
2005,0.806305,0.988,37.058,4247.107,4.210000
2006,1.038258,1.009,44.525,5095.468,2.910000
2007,0.505131,1.103,52.345,5752.749,1.875000


In [ ]:
##